# o-viton
## appearance generation
### test

#### 필요한 패키지 설치

In [1]:
import os
import sys

IS_COLAB = False

if 'COLAB_GPU' in os.environ:
    IS_COLAB=True
else:
    IS_COLAB=False

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [3]:
!pip install dominate
!pip install av
# install dependencies: 
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

1.7.0+cu101 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [4]:
# console에서 다음 작업 진행할 것

!git clone https://github.com/wonyangcho/Image-Based-Virtual-Try-on-Network-from-Unpaired-Data.git /content/o-viton 
# !cd o-vton
# !git checkout develop

fatal: destination path '/content/o-viton' already exists and is not an empty directory.


In [5]:
if IS_COLAB == True:
    from google.colab import drive

In [6]:
!nvidia-smi

Sat Jan 16 16:52:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 300W |     10MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
from pathlib import Path

my_file = Path("/content/dataset")
if my_file.is_dir() == False:
  !wget -O o-viton-dataset.zip https://www.dropbox.com/s/exjzimmtfx6g4be/o-viton-dataset_test.zip?dl=0
  !unzip -o -q o-viton-dataset.zip 
  !rm o-viton-dataset.zip

  !wget -O test_query_ref_label.zip https://www.dropbox.com/s/4mgl14cqi99m0lm/test_query_ref_label.zip?dl=0
  !unzip -o -q test_query_ref_label.zip
  
  !rm test_query_ref_label.zip
  

!ls ./dataset





--2021-01-16 16:52:25--  https://www.dropbox.com/s/exjzimmtfx6g4be/o-viton-dataset_test.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.64.18, 2620:100:6022:18::a27d:4212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.64.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/exjzimmtfx6g4be/o-viton-dataset_test.zip [following]
--2021-01-16 16:52:25--  https://www.dropbox.com/s/raw/exjzimmtfx6g4be/o-viton-dataset_test.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc49f2dd8fd5232d9e004706f765.dl.dropboxusercontent.com/cd/0/inline/BHFF7npJ7dAq6kxioIhdglpaxoHzuc50mdyUZYrule-p7slYuRQ8nyPfxJNeLX9Sux45MWItWVRJY7EuAcBeOZ2-0HMB_C_61A9nemz4LXkiw4gJsUTzChw8OQ1gXHbMDbU/file# [following]
--2021-01-16 16:52:26--  https://uc49f2dd8fd5232d9e004706f765.dl.dropboxusercontent.com/cd/0/inline/BHFF7npJ7dAq6kxioIhdglpaxoHzuc50mdyUZYrule-p7slYuRQ8nyPfxJNeLX9Sux4

#### google drive 연결

In [8]:
if IS_COLAB == True:
    drive.mount('/content/drive')
    !ls "/content/drive/My Drive/02_o-viton/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
source


#### 관련 모듈 불러오기 

#### PATH

In [9]:
if IS_COLAB == True:
  sys.path.append("/content/drive/My Drive/02_o-viton/source/appearance_generation")
  sys.path.append("/content/o-viton/appearance_generation")


In [10]:
import os
from collections import OrderedDict
from torch.autograd import Variable
from options.test_options_colab import TestOptionsColab
from data.ov_test_dataset import TestDataset
from models.models import create_model
import util.util as util
from util.visualizer import Visualizer
from util import html
import torch
from torchvision import transforms
from torch.utils.data import DataLoader

%matplotlib inline

#### chekcpoint경로 확인

In [11]:
!ls /content/drive/MyDrive/02_o-viton/source/appearance_generation/checkpoints

vfr


#### main

In [ ]:
def lcm(a, b): return abs(a * b)/fractions.gcd(a, b) if a and b else 0

def main():
  testOption = TestOptionsColab()
  testOption.initialize()

  opt = testOption.parser

  opt.checkpoints_dir = "/content/drive/My Drive/02_o-viton/source/appearance_generation/checkpoints"
  opt.results_dir = "/content/drive/My Drive/02_o-viton/source/appearance_generation/results"
  opt.dataroot = "/content/dataset"
  opt.nThreads = 1   # test code only supports nThreads = 1
  opt.batchSize = 1  # test code only supports batchSize = 1
  opt.isTrain = False
  

  augment = {}
# augment['1'] = transforms.Compose(
#     [transforms.ToTensor()])  # change to [C, H, W]

  augment['1'] = transforms.Compose(
      [
          transforms.ToTensor(),
          transforms.Normalize((0.5, ), (0.5, ))])

  test_dataset = TestDataset(opt, augment)
  test_dataloader = DataLoader(test_dataset,
                              shuffle=False,
                              num_workers=int(opt.nThreads),
                              batch_size=opt.batchSize,
                              pin_memory=True)

  dataset_size = len(test_dataset)
  print('#testing images = %d' % dataset_size)

  # Create and Load Model
  model = create_model(opt)

  # Initialize visualizer
  visualizer = Visualizer(opt)

  # create website
  web_dir = os.path.join(opt.results_dir, opt.name, '%s_%s' % (opt.phase, opt.which_epoch))
  webpage = html.HTML(web_dir, 'Experiment = %s, Phase = %s, Epoch = %s' % (opt.name, opt.phase, opt.which_epoch))
  for i, data in enumerate(test_dataloader):
      if i >= opt.how_many:
          break
      generated = model.inference_forward_appearance(data['query_img'],data['query_parse_map'],
                                                                data['query_seg_map'],data['ref_img'],
                                                                data['ref_parse_map'],data['ref_seg_map'],
                                                                data['C_tensor_parse_map'],data['C_tensor_seg_map'])
      visuals = OrderedDict([('query_img', util.tensor2im(data['query_img'][0])),
                            ('ref_image', util.tensor2im(data['ref_img'][0])),
                            ('generated_parse_map', util.tensor2label(data['C_tensor_parse_map'][0], opt.label_nc)),
                            ('synthesized_image', util.tensor2im(generated.data[0]))])
      img_path = data['path']
      print('process image... %s' % img_path)
      visualizer.save_images(webpage, visuals, img_path)

  webpage.save()
if __name__ == '__main__':
    # torch.multiprocessing.set_start_method('spawn')
    main()

#testing images = 1016
--------- model used --------- ov_pix2pixHD
GlobalGenerator(
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): Conv2d(50, 64, kernel_size=(7, 7), stride=(1, 1))
    (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (3): ReLU(inplace=True)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (11): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 1024, kernel_size=(3, 3), stride=(2, 2), padding=

/content/o-viton/appearance_generation/models/ov_pix2pixHD_model.py:280: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  num_seg_channel)).nonzero()  # nx4


process image... ['/content/dataset/test_query_label/009809_0.png']
process image... ['/content/dataset/test_query_label/009822_0.png']
process image... ['/content/dataset/test_query_label/009832_0.png']
process image... ['/content/dataset/test_query_label/009843_0.png']
process image... ['/content/dataset/test_query_label/009852_0.png']
process image... ['/content/dataset/test_query_label/009860_0.png']
process image... ['/content/dataset/test_query_label/009872_0.png']
process image... ['/content/dataset/test_query_label/009882_0.png']
process image... ['/content/dataset/test_query_label/009894_0.png']
process image... ['/content/dataset/test_query_label/009903_0.png']
process image... ['/content/dataset/test_query_label/009912_0.png']
process image... ['/content/dataset/test_query_label/009922_0.png']
process image... ['/content/dataset/test_query_label/009932_0.png']
process image... ['/content/dataset/test_query_label/009941_0.png']
process image... ['/content/dataset/test_query_l